In [1]:
import pandas as pd

In [2]:
## dropping status date because I don't know what they mean

In [3]:
def read_tbl():
    df = pd.read_excel("../../../../data/ID/2023-5-12/employee_public_record_report_5-23-20231220144255.xlsx")
    return df 

df = read_tbl()

# Step 1: Assign a sequential unique Person ID 
df['person_nbr'] = pd.factorize(df['First Name'] + df['Last Name'])[0] + 1

duplicate_names = df[df.duplicated(subset=['First Name', 'Last Name'], keep=False)]

# Filter to only keep rows where there are different Person IDs for the same name. There are none. Our sequential ID seems to be unique. 
duplicate_names = duplicate_names.groupby(['First Name', 'Last Name']).filter(lambda x: x['person_nbr'].nunique() > 1)

### proceed with cleaning

def clean_agency(df):
    df.loc[:, "agency_name"] = (df
                           .agency_name
                           .str.lower()
                           .str.strip()
                           .str.replace(r"dept\.", "department", regex=True)
                           .str.replace(r"&", "and", regex=False)
                           .str.replace(r"\bpd\b", "police department", regex=True)
                           .str.replace(r"(\w+) \s+(\w+)", r"\1 \2", regex=True)
    )
    df = df[~((df.agency_name.str.contains("fire")))]
    return df

def rename_cols(df):
    df = df.rename(columns={"First Name": "first_name", "Last Name": "last_name", 
                            "Employment Classification": "employment_classification", 
                            "Start Date": "start_date", "End Date": "end_date", "Start Action": "start_action",
                            "End Action": "separation_reason", "Agency": "agency_name", 
                            "Certification": "certification_type", "Status": "employment_status", 
                            "Issue Date": "certification_date", "Certification Classification": "certification_classification", 
                            "Level": "certification_level", })
    return df.drop(columns=["Archived", "Status Date"])

df = df.pipe(rename_cols).pipe(clean_agency)

df

,first_name,last_name,employment_classification,start_date,end_date,start_action,separation_reason,agency_name,certification_type,employment_status,certification_date,certification_classification,certification_level,person_nbr
0,Keith,Aamodt,Patrol,2017-12-06,2021-07-07,Hired,Resigned,emmett police department,Basic Peace Officer,Inactive,2015-05-26,Patrol,Basic,1
1,Keith,Aamodt,Patrol,2017-12-06,2021-07-07,Hired,Resigned,emmett police department,Intermediate Peace Officer,Inactive,2018-12-19,Patrol,Intermediate,1
2,Keith,Aamodt,Patrol,2017-12-06,2021-07-07,Hired,Resigned,emmett police department,K9 Controlled Substance Detection,Inactive,2019-09-24,K9,Unknown,1
3,Keith,Aamodt,Patrol,2014-01-27,2016-08-29,Hired,Transfer From,kellogg police department,Basic Peace Officer,Inactive,2015-05-26,Patrol,Basic,1
4,Keith,Aamodt,Patrol,2014-01-27,2016-08-29,Hired,Transfer From,kellogg police department,Intermediate Peace Officer,Inactive,2018-12-19,Patrol,Intermediate,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82311,Nicholas,Zweigart,Investigation,2021-04-27,2022-03-09,Transfer To,Transfer From,bannock county sheriff's office,Basic Peace Officer,Active,2008-12-01,Patrol,Basic,13330
82312,Nicholas,Zweigart,Investigation,2021-04-27,2022-03-09,Transfer To,Transfer From,bannock county sheriff's office,K9 Controlled Substance Detection,Expired,2020-04-06,K9,Unknown,13330
82313,Nicholas,Zweigart,Patrol,2022-03-10,NaT,Transfer To,NaN,bannock county sheriff's office,Level I Detention,Active,2005-01-12,Detention,Unknown,13330
82314,Nicholas,Zweigart,Patrol,2022-03-10,NaT,Transfer To,NaN,bannock county sheriff's office,Basic Peace Officer,Active,2008-12-01,Patrol,Basic,13330


In [4]:
df.to_csv("../data/output/idaho_index.csv", index=False)